### 9.3 체인을 활용한 고급 방법
* 영상 튜토리얼 : https://youtu.be/IXTg32-kcFA
* 책 307~312 쪽
* 랭체인 최신 API를 반영하여 책의 API 사용법과 차이가 있습니다.
* 예: run() 대신 invoke() 를 사용하며 RunnableSequence 를 사용합니다.

<img src='https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/cover.png'
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;"   width='300'>


* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

colab 실습 :
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter9_notebooks/Ch9_Advanced_LangChain_Configurations_and_Pipeline.ipynb)  


원서 Colab 실습 :   
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   
<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter9_notebooks/Ch9_Advanced_Methods_with_Chains.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**노트북의 목적:**  
**Langchain**을 활용하여 다음과 같은 방법들을 탐구합니다:  
* LLM을 통한 일반 지식 질문하기  
* LLM 응답으로부터 구조화된 데이터 형식 도출하기  
* 대화 내에서 메모리 기능 설정하기  

**필요 조건:**  
* Colab에서 실행 시 다음 런타임 노트북 설정이 필요: `Python3, CPU`  
* 이 코드는 LLM으로 OpenAI의 API를 사용하므로 유료 **API 키**가 필요합니다.

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

설치:

In [1]:
!pip install -qU langchain openai langchain-openai
!pip install -qU langchain-community

Imports:

In [2]:
import json
import os
import pandas as pd
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.schema.runnable.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_openai import ChatOpenAI

코드 설정:

OpenAI API key:  
**문자열 형태로 아래 "..."에 OpenAI에서 발급받은 key를 입력해 주세요!**  


Colab 보안 비밀 설정은 왼쪽 열쇠 모양의 아이콘을 클릭하면 나옵니다.
<img src="https://i.imgur.com/7P383n4.png" width="500">

유료 LLM이 아닌 무료 LLM을 활용하고자 한다면, 책에서 설명하는 허깅페이스를 활용하는 예제를 따라 대체 방법을 사용해 보세요.

In [3]:
os.environ["OPENAI_API_KEY"] = "..."

# Colab 에서는 보안 비밀키 설정을 통해 API 키를 매번 입력하지 않고 아랴와 같이 관리할 수 있습니다.
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    if not os.environ["OPENAI_API_KEY"]:
        raise ValueError("Colab의 보안 비밀 설정에서 'OPENAI_API_KEY'를 찾을 수 없습니다.")
except ImportError:
    print("Colab 환경이 아니므로, 'api_key' 변수에 직접 값을 할당해야 합니다.")

### LLM에 일반적인 지식 질문하기
* [LangChain Expression Language (LCEL) | 🦜️🔗 LangChain](https://python.langchain.com/docs/concepts/lcel/)

In [4]:
simple_question = "메탈리카(Metallica) 멤버는 누구인가요? 쉼표로 구분해 나열해 주세요. 멤버 이름 외 다른 텍스트는 생성하지 마세요."

chat = ChatOpenAI(model_name="gpt-4o-mini")  # 또는 사용 가능한 다른 ChatGPT 모델
chat_prompt = PromptTemplate.from_template(simple_question)

# RunnableSequence
chain = chat_prompt | chat

result = chain.invoke({})
print(result.content)

제임스 헷필드, 라스 울리히, 키스 해멧, 로라 올리엣


### LLM이 특정 데이터 형식으로 출력을 제공하도록 설정하기

In [5]:
request_list_format = """주기율표에서 처음 10개의 원소를 쉼표로 구분하여 작성해 주세요.
원소 외에 다른 텍스트는 생성하지 마세요."""

prompt = PromptTemplate(template=request_list_format, input_variables=[])

# 출력값을 쉼표로 구분된 리스트로 변환해주는 파서 준비
output_parser = CommaSeparatedListOutputParser()

# 모델과 프롬프트 준비
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

# Runnable Chain 구성 (Prompt → Model → Output Parser)
chain = prompt | chat | output_parser

# 실행
result = chain.invoke({})
print(result)

['수소', '헬륨', '리튬', '베릴륨', '붕소', '탄소', '질소', '산소', '플루오르', '네온']


### 자연스러운 대화를 위해 발전하기
* 이전 상호작용을 참조와 맥락으로 활용할 수 있도록 메모리 요소를 추가해 후속 프롬프트를 이어가는 방식

<img src="https://python.langchain.com/v0.2/assets/images/message_history-4c13b8b9363beb4621d605bf6b5a34b4.png">

* 이미지 출처 : https://python.langchain.com/v0.2/docs/how_to/message_history/

In [6]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 대화형 AI 비서야. 이전 대화를 반드시 참고해서 대답해."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# 세션별 히스토리를 저장할 딕셔너리
session_histories = {}

# 세션별 히스토리 관리 함수
def get_session_history(session_id):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]

# Runnable 구성 (OutputParser 없음)
chain = RunnableWithMessageHistory(
    runnable=prompt | llm,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 첫 번째 입력
first_input = "당신이 알고 있는 대한민국의 공휴일 10가지를 쉼표로 구분하여 나열해 주세요."

first_result = chain.invoke(
    {"input": first_input},
    config={"configurable": {"session_id": "user-1"}}
)

print(f"첫 번째 응답:\n{first_result.content}\n")

첫 번째 응답:
설날, 삼일절, 어린이날, 부처님오신날, 현충일, 광복절, 추석, 개천절, 한글날, 크리스마스.



In [7]:
# 두 번째 입력 (첫 번째 응답 기억)
second_input = "방금 나열한 공휴일 목록에서 종교적인 공휴일을 제거해 주세요."

second_result = chain.invoke(
    {"input": second_input},
    config={"configurable": {"session_id": "user-1"}}
)

print(f"두 번째 응답:\n{second_result.content}\n")

두 번째 응답:
설날, 삼일절, 어린이날, 현충일, 광복절, 추석, 개천절, 한글날.



In [8]:
advanced_data_structure = """각 공휴일에 대해 두 문장으로 설명하세요. 출력은 JSON 형식의 표로 작성해 주세요.
표의 이름은 "holidays"이고, 필드는 "name"과 "description"입니다. 각 행에서 "name"은 공휴일의 이름이고,
"description"은 생성된 설명입니다. 출력의 구문은 줄 바꿈 없이 JSON 형식이어야 합니다.

예시:
{"holidays": [
        {"name": "holiday_name",
         "description": "holiday_description"
        }
        ]}
"""

output = chain.invoke(
    {"input": advanced_data_structure},
    config={"configurable": {"session_id": "user-1"}}
)

print(type(output))

# AIMessage 객체의 content 부분만 추출
response_text = output.content

# JSON 파싱
parsed_json = json.loads(response_text)

# holidays 필드만 추출
holidays = parsed_json.get("holidays", [])

# 결과 출력
print(holidays)


<class 'langchain_core.messages.ai.AIMessage'>
[{'name': '설날', 'description': '설날은 음력 1월 1일로, 한국의 전통적인 새해입니다. 가족들이 모여 차례를 지내고 세배를 하며 새해를 맞이합니다.'}, {'name': '삼일절', 'description': '삼일절은 1919년 3월 1일에 일어난 독립운동을 기념하는 날입니다. 한국의 독립을 위해 싸운 이들을 기리며 각종 기념 행사가 열립니다.'}, {'name': '어린이날', 'description': '어린이날은 5월 5일로, 어린이들의 건강과 행복을 기원하는 날입니다. 부모들은 아이들과 함께 시간을 보내며 다양한 활동을 즐깁니다.'}, {'name': '현충일', 'description': '현충일은 6월 6일로, 나라를 위해 목숨을 바친 순국선열과 호국영령을 기리는 날입니다. 전국적으로 묵념과 추모 행사가 진행됩니다.'}, {'name': '광복절', 'description': '광복절은 8월 15일로, 1945년 일본의 식민지 지배로부터 해방된 것을 기념하는 날입니다. 이 날을 기념하여 다양한 축하 행사가 열립니다.'}, {'name': '추석', 'description': '추석은 음력 8월 15일로, 한국의 가을 수확을 감사하는 명절입니다. 가족들이 모여 성묘를 하고 송편을 나누어 먹습니다.'}, {'name': '개천절', 'description': '개천절은 10월 3일로, 단군이 고조선을 건국한 것을 기념하는 날입니다. 한국의 건국 신화를 기리는 다양한 행사가 열립니다.'}, {'name': '한글날', 'description': '한글날은 10월 9일로, 세종대왕이 훈민정음을 반포한 것을 기념하는 날입니다. 한글의 우수성을 기리고 보존하기 위한 행사가 진행됩니다.'}]


In [12]:
pd.set_option('display.max_colwidth', None)

dict_content = json.loads(output.content)
pd.DataFrame(dict_content['holidays'])

,name,description
0,설날,"설날은 음력 1월 1일로, 한국의 전통적인 새해입니다. 가족들이 모여 차례를 지내고 세배를 하며 새해를 맞이합니다."
1,삼일절,삼일절은 1919년 3월 1일에 일어난 독립운동을 기념하는 날입니다. 한국의 독립을 위해 싸운 이들을 기리며 각종 기념 행사가 열립니다.
2,어린이날,"어린이날은 5월 5일로, 어린이들의 건강과 행복을 기원하는 날입니다. 부모들은 아이들과 함께 시간을 보내며 다양한 활동을 즐깁니다."
3,현충일,"현충일은 6월 6일로, 나라를 위해 목숨을 바친 순국선열과 호국영령을 기리는 날입니다. 전국적으로 묵념과 추모 행사가 진행됩니다."
4,광복절,"광복절은 8월 15일로, 1945년 일본의 식민지 지배로부터 해방된 것을 기념하는 날입니다. 이 날을 기념하여 다양한 축하 행사가 열립니다."
5,추석,"추석은 음력 8월 15일로, 한국의 가을 수확을 감사하는 명절입니다. 가족들이 모여 성묘를 하고 송편을 나누어 먹습니다."
6,개천절,"개천절은 10월 3일로, 단군이 고조선을 건국한 것을 기념하는 날입니다. 한국의 건국 신화를 기리는 다양한 행사가 열립니다."
7,한글날,"한글날은 10월 9일로, 세종대왕이 훈민정음을 반포한 것을 기념하는 날입니다. 한글의 우수성을 기리고 보존하기 위한 행사가 진행됩니다."
